## Consulta 03

Esta consulta devuelve devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Endpoint: def UserForGenre( genero : str ):\
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



*Importamos la librería y módulo necesarios para el procesamiento de los datos*

In [3]:
import pandas as pd
import csv

*Importamos los archivos con los que vamos a trabajar la consulta*

In [4]:
df_games = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\games_clean.parquet')
df_items = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\user_items_clean.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [5]:
df_games_copy = df_games.copy()
df_items_copy = df_items.copy()

*(Opcional). Podemos hacer una visualización previa del dataset para hacer cualquier análsis previo.*

In [6]:
df_games_copy

,id,app_name,developer,genres,price,release_date
0,761140,Lost Summoner Kitty,Kotoshiro,Action,4.99,2018
1,643980,Ironbound,Secret Level SRL,Indie,0.00,2018
2,670290,Real Pool 3D - Poolians,Poolians.com,Casual,0.00,2017
3,767400,弹炸人2222,彼岸领域,Action,0.99,2017
4,773570,Log Challenge,None,Action,2.99,None
...,...,...,...,...,...,...
32130,773640,Colony On Mars,"Nikita ""Ghost_RUS""",Casual,1.99,2018
32131,733530,LOGistICAL: South Africa,Sacada,Casual,4.99,2018
32132,610660,Russian Roads,Laush Dmitriy Sergeevich,Indie,1.99,2018
32133,658870,EXIT 2 - Directions,"xropi,stev3ns",Casual,4.99,2017


In [7]:
df_items_copy

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


*Desarrollamos la función correspondiente para hacer la consulta que necesitamos*

In [16]:
def UserForGenre(genero: str, df_games_copy, df_items_copy):
    # Verificar si la columna 'genre' está presente en el DataFrame df_games_copy
    if 'genres' not in df_games_copy.columns:
        raise ValueError("El DataFrame df_games_copy no tiene una columna llamada 'genre'.")

    # Convertir la columna 'release_date' a tipo datetime y extraer el año
    df_games_copy['Año'] = pd.to_datetime(df_games_copy['release_date'], errors='coerce').dt.year

    # Filtrar df_games_copy por el género dado
    juegos_genero = df_games_copy[df_games_copy['genres'] == genero]

    # Unir el DataFrame filtrado con df_user_items
    juegos_usuario = juegos_genero.merge(df_items_copy, left_on='id', right_on='item_id')

    # Calcular las horas jugadas por usuario para cada juego
    horas_por_usuario = juegos_usuario.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Verificar si hay datos para el género dado
    if horas_por_usuario.empty:
        return {"Mensaje": f"No hay datos para el género '{genero}'."}

    # Encontrar el usuario con más horas jugadas
    usuario_max_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax()]['user_id']

    # Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado
    horas_por_año = juegos_usuario.groupby('Año')['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'playtime_forever': 'Horas'}, inplace=True)
    horas_por_año['Año'] = horas_por_año['Año'].astype(int)  # Convertir el año a número entero
    horas_por_año = horas_por_año.to_dict('records')

    # Crear el diccionario de resultados
    result = {
        "Usuario con más horas jugadas para {}: ".format(genero): usuario_max_horas,
        "Horas jugadas": horas_por_año
    }

    return result

*Aplicamos la función y corroboramos que funcione correctamente*

In [18]:
#Ejemplo de uso:
resultado = UserForGenre("Simulation", df_games_copy, df_items_copy)
print(resultado)

{'Usuario con más horas jugadas para Simulation: ': '76561198038513068', 'Horas jugadas': [{'Año': 1987, 'Horas': 0.0}, {'Año': 1988, 'Horas': 1.0}, {'Año': 1989, 'Horas': 0.0}, {'Año': 1990, 'Horas': 878.0}, {'Año': 1991, 'Horas': 144.0}, {'Año': 1992, 'Horas': 11.0}, {'Año': 1994, 'Horas': 4127.0}, {'Año': 1995, 'Horas': 980.0}, {'Año': 1998, 'Horas': 4361.0}, {'Año': 1999, 'Horas': 130537.0}, {'Año': 2000, 'Horas': 12342.0}, {'Año': 2001, 'Horas': 215993.0}, {'Año': 2002, 'Horas': 294785.0}, {'Año': 2003, 'Horas': 1363396.0}, {'Año': 2004, 'Horas': 101930.0}, {'Año': 2005, 'Horas': 80704.0}, {'Año': 2006, 'Horas': 2691671.0}, {'Año': 2007, 'Horas': 172230.0}, {'Año': 2008, 'Horas': 3798669.0}, {'Año': 2009, 'Horas': 13880055.0}, {'Año': 2010, 'Horas': 1124125.0}, {'Año': 2011, 'Horas': 3371092.0}, {'Año': 2012, 'Horas': 369703.0}, {'Año': 2013, 'Horas': 16534471.0}, {'Año': 2014, 'Horas': 3738916.0}, {'Año': 2015, 'Horas': 9669002.0}, {'Año': 2016, 'Horas': 5873778.0}, {'Año': 2017,